In [1]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd

In [2]:
def calculate_angle_difference(current_angles, last_saved_angles):
    return sum(abs(current_angles - last_saved_angles))

def process_image_folder(folder_path, threshold):
    csv_file_path = os.path.join(folder_path, 'data.csv')
    output_folder = os.path.join(folder_path, 'processed_video')
    depth_output_folder = os.path.join(folder_path, 'processed_depth')

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    else:
        shutil.rmtree(output_folder)
        os.makedirs(output_folder)
        
    if not os.path.exists(depth_output_folder):
        os.makedirs(depth_output_folder)
    else:
        shutil.rmtree(depth_output_folder)
        os.makedirs(depth_output_folder)
    #     return
    print(output_folder)

    # Read CSV
    df = pd.read_csv(csv_file_path, header=0)
    # last_saved_angles = np.array(df[["RightWristX", "RightWristY"]].iloc[0, :])
    columns = ["LeftShoulderX", "LeftShoulderY", "LeftElbowX", "LeftElbowY", "LeftWristX", "LeftWristY"]
    last_saved_angles = np.array(df[columns].iloc[0, :])
    frame_count = 1
    updated_rows = []

    for i in range(len(df)):
        image_path = os.path.join(folder_path, 'color', f'color_data_{i + 1}.bin')
        if not os.path.exists(image_path):
            break

        with open(image_path, 'rb') as f:
            color_data = f.read()
            color_array = np.frombuffer(color_data, dtype=np.uint8)
            frame = color_array.reshape((-1, 4))
            width = 640
            height = 480
            frame = frame.reshape((height, width, 4))
        # current_angles = np.array(df[["RightWristX", "RightWristY"]].iloc[i, :])
        current_angles = np.array(df[columns].iloc[i, :])
        # current_angles = np.array(df[["RightElbowYaw","RightElbowRoll", "RightShoulderRoll", "RightShoulderPitch"]].iloc[i, :])

        if i == 0 or calculate_angle_difference(current_angles, last_saved_angles) > threshold:
            depth_path = os.path.join(folder_path, 'depth', f'depth_data_{i + 1}.bin')
            cv2.imwrite(f'{output_folder}/frame_{frame_count}.jpg', frame)
            shutil.copy(depth_path, os.path.join(depth_output_folder, f'depth_data_{frame_count}.bin'))
            last_saved_angles = current_angles
            updated_row = df.iloc[i].copy()
            updated_row['FrameNumber'] = frame_count
            updated_rows.append(updated_row)
            frame_count += 1

    # Save the updated CSV
    updated_df = pd.DataFrame(updated_rows)
    updated_df.to_csv(os.path.join(folder_path, 'processed_step_1.csv'), index=False)

In [3]:
# Parameters
threshold = 0.000005
base_folder = r'.\NaoPickAndPlaceData'

# List all directories in the base folder
for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    if os.path.isdir(folder_path) and 'data.csv' in os.listdir(folder_path):
        process_image_folder(folder_path, threshold)

C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-54-15\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-54-59\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-55-33\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-56-28\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-57-47\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-58-21\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_16-59-16\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_17-00-02\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_18-32-01\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_18-33-03\processed_video
C:\Users\ARB\PycharmProjects\Nao\NaoPickAndPlaceData\2024-08-09_18-33-49\processed_video
C:\Users\ARB\PycharmP